# Loan recommendation using Collaborative Filtering

The rating data is in loan_rating.csv, containing lender_id, loan_id, rating. Rating can be from 0-10.

In [76]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
locale.setlocale(locale.LC_ALL, '')

os.chdir(r"C:/My Courses/MATH 5671 - Financial Data Mining and Big Data Analytics/Fall 18/assignments/assignment09/")

loan_rating_df = pd.read_csv("loan_rating.csv",header =0)
loan_rating_df.head()

,lender_id,loan_id,rating
0,0,0,3
1,1,0,0
2,2,0,0
3,3,0,0
4,4,0,2


Use surprise package to read rating data in, train and test the model

In [84]:
loan_rating_df = pd.read_csv("loan_rating.csv")

reader = Reader(rating_scale=(0, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(loan_rating_df[['lender_id', 'loan_id', 'rating']], reader)

In [85]:
from surprise import NMF, evaluate
from surprise import accuracy
from surprise.model_selection import train_test_split

algo = NMF()
trainset, testset = train_test_split(data, test_size=.3)

output = algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 3.4662


3.466205894903756

In [86]:
predictions[:10]

[Prediction(uid=3, iid=27593, r_ui=0.0, est=1.6964805816427302, details={'was_impossible': False}),
 Prediction(uid=2, iid=59769, r_ui=0.0, est=4.784075896384685, details={'was_impossible': False}),
 Prediction(uid=8, iid=83079, r_ui=6.0, est=0.47726630608796095, details={'was_impossible': False}),
 Prediction(uid=6, iid=54236, r_ui=2.0, est=3.016932217007822, details={'was_impossible': False}),
 Prediction(uid=9, iid=81930, r_ui=6.0, est=2.7794575985657746, details={'was_impossible': False}),
 Prediction(uid=7, iid=83933, r_ui=0.0, est=1.8110610608928837, details={'was_impossible': False}),
 Prediction(uid=1, iid=37887, r_ui=0.0, est=1.3227112992914636, details={'was_impossible': False}),
 Prediction(uid=3, iid=67526, r_ui=4.0, est=3.877894189479651, details={'was_impossible': False}),
 Prediction(uid=6, iid=27897, r_ui=3.0, est=1.695124892794956, details={'was_impossible': False}),
 Prediction(uid=4, iid=30563, r_ui=0.0, est=1.6338839141616348, details={'was_impossible': False})]

In [123]:
predictions[0].uid

3

In [146]:
pred_loans = np.array([[pred.iid,pred.est] for pred in predictions if pred.uid == 1])
pred_top_loans_index = np.argsort(pred_loans[:,1])[::-1][:10]
pred_top_loans = pred_loans[pred_top_loans_index, 0]
pred_top_loans

array([46241., 34164.,  1405., 40760., 55661., 59149., 63907., 32441.,
       45801., 40807.])

In [87]:
loan_df = pd.read_csv("loan.csv",header = 0)
loan_df.head()

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
0,5000,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,0.0,83.7,9.0,0,26.0,verified
1,2500,60 months,15.27,0.0,RENT,30000.0,car,GA,1.00,0.0,9.4,4.0,1,12.0,verified
2,2400,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,0.0,98.5,10.0,0,10.0,not verified
3,10000,36 months,13.49,10.0,RENT,49200.0,other,CA,20.00,0.0,21.0,37.0,0,15.0,verified
4,5000,36 months,7.90,3.0,RENT,36000.0,wedding,AZ,11.20,0.0,28.3,12.0,0,7.0,verified


In [152]:
info = loan_df.loc[1,:]
info

loan_amnt                     2500
term                     60 months
int_rate                     15.27
emp_length                       0
home_ownership                RENT
annual_inc                   30000
purpose                        car
addr_state                      GA
dti                              1
delinq_2yrs                      0
revol_util                     9.4
total_acc                        4
bad_loan                         1
longest_credit_length           12
verification_status       verified
Name: 1, dtype: object

Now give recommendations

In [163]:
top = 3
for j in range(n_lenders):
    # get top 3 loans
    # iid is the loan_id 
    pred_loans = np.array([[pred.iid,pred.est] for pred in predictions if pred.uid == j])
    pred_top_loans_index = np.argsort(pred_loans[:,1])[::-1][:top]
    pred_top_loans = pred_loans[pred_top_loans_index, :]
     
    for i in range(limit):
        loan = pred_top_loans[i,:]
        info = loan_df.loc[loan[0],:]
        print('Predicted rating %.1f for loan of  %s  for %s with %s purpose at %.1f percent interest\n' %
        (pred_top_loans[i][1], locale.currency(np.float(info.loan_amnt),grouping=True), info.term, info.purpose, np.float(info.int_rate)))

Predicted rating 7.9 for loan of  $7,000.00  for 60 months with debt_consolidation purpose at 18.0 percent interest

Predicted rating 7.5 for loan of  $15,000.00  for 36 months with debt_consolidation purpose at 8.9 percent interest

Predicted rating 7.1 for loan of  $10,000.00  for 36 months with debt_consolidation purpose at 16.3 percent interest

Predicted rating 8.9 for loan of  $12,000.00  for 36 months with home_improvement purpose at 11.0 percent interest

Predicted rating 7.7 for loan of  $5,000.00  for 36 months with small_business purpose at 9.6 percent interest

Predicted rating 7.6 for loan of  $9,000.00  for 36 months with credit_card purpose at 12.7 percent interest

Predicted rating 10.0 for loan of  $15,000.00  for 36 months with debt_consolidation purpose at 12.1 percent interest

Predicted rating 10.0 for loan of  $3,600.00  for 36 months with moving purpose at 7.0 percent interest

Predicted rating 10.0 for loan of  $23,500.00  for 36 months with debt_consolidation p

In [90]:
n_lenders = len(loan_rating_df['lender_id'].unique())
n_loans = len(loan_rating_df['loan_id'].unique())

In [112]:
loan_rating_df[loan_rating_df['loan_id'] == 100][loan_rating_df['lender_id'] == 1][['rating']].values[0,0]

0

Now a recommender system model from scratch

In [166]:
from sklearn.model_selection import train_test_split
import numpy as np
import random
import csv
from sklearn.model_selection import train_test_split
import locale # for currency formatting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

loan_rating_df = pd.read_csv("loan_rating.csv",header =0)
loan_rating_df.head()

In [257]:
lenders = loan_rating_df['lender_id'].unique()
n_lenders = len(lenders)
loans = loan_rating_df['loan_id'].unique()
n_loans = len(loans)
Y = np.empty([n_loans, n_lenders])

for loan in loans[:limit]:
    for lender in lenders:
        rating = loan_rating_df[loan_rating_df['loan_id'] == loan][loan_rating_df['lender_id'] == lender]['rating']
        if len(rating) == 0:
            Y[loan,lender] = -1
        else:
            Y[loan,lender] = rating.values[0]
        
        Y = Y.astype(float)

# use just part of Y (like, 100 loans)
limit = 100
Y = Y[:limit,:]
R = (Y != -1)
R = R.astype(float)

In [258]:
loan_df = pd.read_csv("loan.csv",header =0)
loan_df.head()
info = np.array(loan_df)[:limit,:]

Starting here write your own code in these functions

In [ ]:
def costFun(param, Y, r, n_lenders, n_loans, n_features, lamba):
    # Write your code here
    
    return J

def cost_grad(param, Y, r, n_lenders, n_loans, n_features, lamba):    
    # Write your code here
    
    return grad
    
def optimizeCost(param, Y, r, n_lenders, n_loans, n_features, lamba, step, maxrun):
    # Write your code here
    
    return param,cost_range


n_lenders = np.size(Y, 1)
n_loans = np.size(Y, 0)
n_features = 10
# Initialization
X = np.random.normal(loc = 0.0, scale = 1.0, size = (n_loans, n_features))
Theta = np.random.normal(loc = 0.0, scale = 1.0, size = (n_lenders,n_features))
init_param = np.concatenate((X.reshape(n_loans * n_features, 1, order = "F"),
                             Theta.reshape(n_lenders * n_features, 1, order = "F")))
init_param = np.squeeze(init_param)

# Optimization
lamba = 10
maxrun = 10000
step = 0.001

param,cost_range = optimizeCost(init_param, Y, R, n_lenders, n_loans, \
                     n_features, lamba, step, maxrun)
# now plot the cost
plt.plot(cost_range,"b.",markersize=1,label="Cost") # note: this is 0-based
plt.show()
    
# Extract X and Theta from param vector
X = param[0:(n_loans * n_features)]
Theta = param[(n_loans * n_features):len(param)]
X = X.reshape(n_loans, n_features, order = "F")
Theta = Theta.reshape(n_lenders, n_features, order = "F")
pred = np.dot(X, Theta.T)

# print out top 3 ratings for each lender
top_n = 3
for j in range(n_lenders):
    rating = np.sort(pred[:, j])[::-1]
    ind = np.argsort(pred[:, j])[::-1]
    a = info[ind,:]
    print('\nTop %d recommendations for lender %d:\n' % (top_n, (j+1)))
    for i in range(top_n):
        print('Predicted rating %.1f for loan of  %s  for %s with %s purpose at %.1f percent interest\n' %
        (rating[i], locale.currency(np.float(a[i, 0]),grouping=True), a[i, 1], a[i, 6], np.float(a[i,2])))  